# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [5]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [13]:
def oracle010(qq):
    yield X(qq[0])
    yield X(qq[2])
    yield Z(qq[1]).controlled_by(qq[0],qq[2])
    yield X(qq[0])
    yield X(qq[2])



In [14]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [15]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [16]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [17]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 95, '110': 3, '100': 1, '001': 1})


95

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla. 

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [18]:
def oracle2(qq):

    def edge_check(a, b, c):
        yield cirq.CNOT(qq[a], qq[c])
        yield cirq.CNOT(qq[b], qq[c])

    yield edge_check(0, 3, 5)
    yield edge_check(0, 4, 6)
    yield edge_check(1, 3, 7)
    yield edge_check(1, 4, 8)
    yield edge_check(2, 3, 9)
    yield edge_check(2, 4, 10)

    yield cirq.X(qq[11]).controlled_by(*qq[5:11])



In [19]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [20]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [21]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'00011': 465,
         '11100': 440,
         '11001': 7,
         '01100': 5,
         '10001': 5,
         '01000': 5,
         '00111': 5,
         '00000': 5,
         '01010': 4,
         '00010': 4,
         '00101': 4,
         '10101': 4,
         '01111': 4,
         '10011': 3,
         '01101': 3,
         '10010': 3,
         '11010': 3,
         '00001': 3,
         '10000': 3,
         '11101': 3,
         '11011': 3,
         '11000': 2,
         '10111': 2,
         '00100': 2,
         '01110': 2,
         '01001': 2,
         '11111': 2,
         '01011': 2,
         '11110': 2,
         '10110': 2,
         '10100': 1})

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [22]:
def oracle3(qq):
    
    yield CX(qq[0], qq[4])
    yield CX(qq[1], qq[4])

    yield CX(qq[0], qq[5])
    yield CX(qq[2], qq[5])

    yield CX(qq[0], qq[6])
    yield CX(qq[3], qq[6])

    yield CX(qq[1], qq[7])
    yield CX(qq[2], qq[7])

    yield CX(qq[1], qq[8])
    yield CX(qq[3], qq[8])

    # check all edge qubits
    yield X(qq[12]).controlled_by(*(qq[4:9]))

In [23]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [24]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [25]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1101': 73,
         '0001': 70,
         '0010': 69,
         '1001': 67,
         '0100': 66,
         '1011': 65,
         '0000': 64,
         '0111': 64,
         '1111': 63,
         '0101': 61,
         '0011': 61,
         '1100': 60,
         '0110': 57,
         '1110': 57,
         '1010': 53,
         '1000': 50})

In [ ]:
# hidden tests in this cell will be used for grading.